In [14]:
# kernel must be msd2
import pandas as pd
import constant as c
import geopandas as gpd
import numpy as np
from shapely.wkt import loads

# Graphics
import matplotlib.pyplot as plt
import seaborn
from pysal.viz import splot
from splot.esda import plot_moran

import contextily as cx
# Analysis
import geopandas
import pandas
from pysal.explore import esda
from pysal.lib import weights
from numpy.random import seed

from esda.moran import Moran
from libpysal.weights import Queen, KNN
import seaborn
import pandas
import geopandas
import numpy
import matplotlib.pyplot as plt

from pyproj import CRS
import warnings; warnings.filterwarnings('ignore')

import geopandas as gpd
from libpysal.weights import Queen
from esda.moran import Moran_Local
from libpysal.weights import full

# then read the data
df_2020 = pd.read_csv('../asset/preprocess/df_2020.csv', index_col=0)
df_2020['geometry_grids'] = df_2020['geometry_grids'].apply(lambda x: loads(x))
df_2021 = pd.read_csv('../asset/preprocess/df_2021.csv', index_col=0)
df_2021['geometry_grids'] = df_2021['geometry_grids'].apply(lambda x: loads(x))
df_2022 = pd.read_csv('../asset/preprocess/df_2022.csv', index_col=0)
df_2022['geometry_grids'] = df_2022['geometry_grids'].apply(lambda x: loads(x))

crs = 'EPSG:5179'  # Specify the coordinate reference system
gdf_2020 = gpd.GeoDataFrame(df_2020, geometry=df_2020['geometry_grids'], crs=crs)
gdf_2021 = gpd.GeoDataFrame(df_2021, geometry=df_2021['geometry_grids'], crs=crs)
gdf_2022 = gpd.GeoDataFrame(df_2022, geometry=df_2022['geometry_grids'], crs=crs)

gdf_2020_scbd = gdf_2020[gdf_2020['sigungunm'].isin(c.SCBD_NMS)]
gdf_2020_gbd = gdf_2020[gdf_2020['sigungunm'].isin(c.GBD_NMS)]
gdf_2020_ybd = gdf_2020[gdf_2020['sigungunm'].isin(c.YBD_NMS)]
gdf_2021_scbd = gdf_2021[gdf_2021['sigungunm'].isin(c.SCBD_NMS)]
gdf_2021_gbd = gdf_2021[gdf_2021['sigungunm'].isin(c.GBD_NMS)]
gdf_2021_ybd = gdf_2021[gdf_2021['sigungunm'].isin(c.YBD_NMS)]
gdf_2022_scbd = gdf_2022[gdf_2022['sigungunm'].isin(c.SCBD_NMS)]
gdf_2022_gbd = gdf_2022[gdf_2022['sigungunm'].isin(c.GBD_NMS)]
gdf_2022_ybd = gdf_2022[gdf_2022['sigungunm'].isin(c.YBD_NMS)]


In [17]:
# Assuming you have the necessary data for the GeoDataFrame
gdf = gpd.GeoDataFrame(gdf_2022_ybd, geometry='geometry_grids')

# Set the CRS for the GeoDataFrame
crs = CRS.from_epsg(5179)
gdf = gdf.set_crs(crs)
# gdf.explore(column="bs_ebit", zoom_start=13, width=1000, height=600)
gdf

,grid_idx,pp_pop,pp_od,bs_ebit,geometry_grids,sigungunm,platplc,x,y,x_5179,...,bd_totarea,bd_totflrcnt,bd_elvtent,bs_gas,bs_elct,bd_height,bd_ilp,bd_vintage,count,geometry
0,10718,105816.180846,2.619875e+06,3.158297e+10,"POLYGON ((945535.230 1946565.548, 945535.230 1...",영등포구,서울특별시 영등포구 문래동5가 23-5번지,126.883851,37.517448,945550.923487,...,14895.98,25,1.500000,220746.0,75494.0,57.125,4.634000e+06,18.000000,4,"POLYGON ((945535.230 1946565.548, 945535.230 1..."
1,10719,104955.254678,2.596789e+06,3.249659e+10,"POLYGON ((945535.230 1946665.548, 945535.230 1...",영등포구,서울특별시 영등포구 문래동5가 23-2번지,126.883935,37.517784,945558.596665,...,1304.02,4,1.000000,0.0,0.0,13.500,4.705000e+06,7.000000,4,"POLYGON ((945535.230 1946665.548, 945535.230 1..."
2,10739,106532.399176,2.728659e+06,3.142737e+10,"POLYGON ((945635.230 1946365.548, 945635.230 1...",영등포구,서울특별시 영등포구 문래동5가 20-20번지,126.885534,37.515291,945698.098718,...,7838.89,65,0.666667,190367.0,31871.0,20.000,4.491111e+06,11.777778,4,"POLYGON ((945635.230 1946365.548, 945635.230 1..."
3,10745,103451.477088,2.614339e+06,3.848601e+10,"POLYGON ((945635.230 1946965.548, 945635.230 1...",영등포구,서울특별시 영등포구 문래동6가 34-1번지,126.885623,37.520634,945709.807788,...,2123.84,15,1.000000,101966.0,29202.0,43.700,5.553000e+06,4.000000,4,"POLYGON ((945635.230 1946965.548, 945635.230 1..."
4,10764,107573.784604,2.819698e+06,3.221564e+10,"POLYGON ((945735.230 1946265.548, 945735.230 1...",영등포구,서울특별시 영등포구 문래동5가 5-4번지,126.886766,37.514684,945806.565887,...,486.72,2,0.000000,0.0,0.0,12.000,4.991000e+06,20.000000,3,"POLYGON ((945735.230 1946265.548, 945735.230 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,12650,105175.260278,2.671067e+06,7.047736e+12,"POLYGON ((949835.230 1946565.548, 949835.230 1...",영등포구,서울특별시 영등포구 여의도동 54-7번지,126.932886,37.517758,949884.450470,...,828.85,6,0.000000,20903.0,3998.0,13.600,1.636000e+07,47.000000,9,"POLYGON ((949835.230 1946565.548, 949835.230 1..."
646,12677,101771.757407,2.613243e+06,7.421712e+12,"POLYGON ((949935.230 1946665.548, 949935.230 1...",영등포구,서울특별시 영등포구 여의도동 53-3번지,126.933547,37.518740,949943.459221,...,1991.49,8,1.000000,0.0,29517.0,25.200,1.514000e+07,38.000000,9,"POLYGON ((949935.230 1946665.548, 949935.230 1..."
647,12702,107150.844889,2.577036e+06,6.513553e+12,"POLYGON ((950035.230 1946565.548, 950035.230 1...",영등포구,서울특별시 영등포구 여의도동 62번지,126.935605,37.517823,950124.743329,...,2272.57,9,0.500000,1633182.0,1026125.0,37.640,1.475500e+07,9.500000,9,"POLYGON ((950035.230 1946565.548, 950035.230 1..."
648,12778,91857.671802,2.475424e+06,6.681752e+12,"POLYGON ((950335.230 1946765.548, 950335.230 1...",영등포구,서울특별시 영등포구 여의도동 61-4번지,126.939026,37.518953,950427.839088,...,59362.35,29,5.000000,99424.0,216623.0,88.180,1.802000e+07,28.000000,9,"POLYGON ((950335.230 1946765.548, 950335.230 1..."


In [4]:
# for ybd append 12528(유진빌딩) grid because centroid(유진빌딩) has no neighbors
ybd_initial_grid_idx = [12556, 12481, 12528, 12503]
init_w = Queen.from_dataframe(gdf_2022_ybd)

In [20]:
indexes = gdf_2022_ybd[gdf_2022_ybd['grid_idx'].isin([12556, 12481, 12528, 12503])].index

# Print the indexes
print(indexes)

Int64Index([620, 626, 633, 638], dtype='int64')


In [19]:
# if initial centroid lacks neighbor that locates directly to the next, we set neighbor as the nearest neighbor.

Centroid Building is 파크원
gdf_idx is 638
w_idx is 638
[620, 626, 633, 638] [620, 626, 633, 638]


In [33]:
# define incremental sar target gdf
gdf = gdf_2022_ybd.copy()

# ybd
first_gdf_idx    = list(gdf.index)[0] # 650(for gbd)
centroid_gdf_idx = gdf['bd_height'].idxmax()
centroid_w_idx = centroid_gdf_idx - first_gdf_idx
print(f'Centroid Building is {gdf.loc[centroid_gdf_idx].bdnm}\ngdf_idx is {centroid_gdf_idx}\nw_idx is {centroid_w_idx}')
init_w = Queen.from_dataframe(gdf)

# if initial centroid lacks neighbor that locates directly to the next, we set neighbor as the nearest neighbor.
init_gdf_idxs = list(gdf_2022_ybd[gdf_2022_ybd['grid_idx'].isin([12556, 12481, 12528, 12503])].index)
init_w_idxs = [x - first_gdf_idx for x in list(init_gdf_idxs)]

print(init_gdf_idxs, init_w_idxs)
cluster = init_w_idxs # we manage by gdf_idx, not w_idx because w_idx changes everytime

prev_w_idxs = cluster
for phase in range(16):
    tba_w_idxs = []
    for w_idx in prev_w_idxs:
        tba_idxs = init_w.neighbors[w_idx]
        tba_w_idxs.extend(tba_idxs) # tba: to be added
    assert len(tba_w_idxs) > 0
    cluster.extend(tba_w_idxs)
    cluster = list(set(cluster))
    cluster_gdf_idxs = [x + first_gdf_idx for x in list(cluster)]
    if phase % 2 == 0: # execute test for every 3 phases
        new_gdf = gdf.loc[cluster_gdf_idxs]
        new_w = Queen.from_dataframe(new_gdf)
        new_w.transform = 'r'
        moran = Moran(new_gdf['bd_totarea'], new_w)
        z = moran.z_sim
        print('bdnms', new_gdf.bdnm.values)
        print('round', phase)
        print('z', z)

Centroid Building is 파크원
gdf_idx is 638
w_idx is 638
[620, 626, 633, 638] [620, 626, 633, 638]
('WARNING: ', 10, ' is an island (no neighbors)')
bdnms ['여의도순복음교회' '(주)유수홀딩스, 현대차증권빌딩' 'BNK금융타워' 'C.C.M.M빌딩'
 '(주)신한 여의도 드림리버 오피스텔' '신송빌딩, 메리츠화재여의도사옥' '서울 국제금융 센터' '여의도 태영빌딩, 세우빌딩'
 'NoData' '유진빌딩' '파크원']
round 0
z 0.45772885971551974
('WARNING: ', 17, ' is an island (no neighbors)')
bdnms ['S-TRENUE' '여의도파이낸스타워, 유화증권빌딩, 오투타워' '하이투자증권, 신한투자증권타워' '여의도순복음교회'
 '여의도순복음교회' '교보증권빌딩' '(주)유수홀딩스, 현대차증권빌딩' 'BNK금융타워' 'C.C.M.M빌딩'
 '(주)신한 여의도 드림리버 오피스텔' '한국가정법률상담소' '신송센타빌딩' '신송빌딩, 메리츠화재여의도사옥'
 '서울 국제금융 센터' '여의도 태영빌딩, 세우빌딩' 'NoData' '유진빌딩' '파크원']
round 2
z 0.9942429226847047
('WARNING: ', 17, ' is an island (no neighbors)')
bdnms ['S-TRENUE' '여의도파이낸스타워, 유화증권빌딩, 오투타워' '하이투자증권, 신한투자증권타워' '여의도순복음교회'
 '여의도순복음교회' '교보증권빌딩' '(주)유수홀딩스, 현대차증권빌딩' 'BNK금융타워' 'C.C.M.M빌딩'
 '(주)신한 여의도 드림리버 오피스텔' '한국가정법률상담소' '신송센타빌딩' '신송빌딩, 메리츠화재여의도사옥'
 '서울 국제금융 센터' '여의도 태영빌딩, 세우빌딩' 'NoData' '유진빌딩' '파크원']
round 4
z 0.9439568715756474
('W

In [36]:
gdf.loc[cluster_gdf_idxs].explore()

In [2]:
def incremental_sar(gdf, initial_cluster_gdf_idx):
    first_gdf_idx    = list(gdf.index)[0] # 650(for gbd)
    centroid_gdf_idx = gdf['bd_height'].idxmax()
    centroid_w_idx = centroid_gdf_idx - first_gdf_idx
    print(f'Centroid Building is {gdf.loc[centroid_gdf_idx].bdnm}\ngdf_idx is {centroid_gdf_idx}\nw_idx is {centroid_w_idx}')
    init_w = Queen.from_dataframe(gdf)
    cluster = [centroid_w_idx] # we manage by gdf_idx, not w_idx because w_idx changes everytime
    for phase in range(50):
        if phase == 0:
            tba_w_idxs = init_w.neighbors[centroid_w_idx]
            print(tba_w_idxs)
        else :
            prev_w_idx = tba_w_idxs
            tba_w_idxs = []
            for w_idx in prev_w_idx:
                tba_idxs = init_w.neighbors[w_idx]
                tba_w_idxs.extend(tba_idxs) # tba: to be added
        assert len(tba_w_idxs) > 0
        cluster.extend(tba_w_idxs)
        cluster = list(set(cluster))
        cluster_gdf_idxs = [x + first_gdf_idx for x in list(cluster)]
        if phase % 3 == 0: # execute test for every 3 phases
            new_gdf = gdf.loc[cluster_gdf_idxs]
            new_w = Queen.from_dataframe(new_gdf)
            new_w.transform = 'r'
            moran = Moran(new_gdf['bd_ilp'], new_w)
            z = moran.z_sim
            print('bdnms', new_gdf.bdnm.values)
            print('round', phase)
            print('z', z)
                
incremental_sar(gdf_2022_ybd)   

Centroid Building is 파크원
gdf_idx is 638
w_idx is 638
[]


AssertionError: 

In [ ]:
def calculate_similarity(gdf, cluster, w):
    similarity_scores = {}
    for grid_id in gdf.index:
        if grid_id in cluster:
            similarity_scores[grid_id] = 1.0
        else:
            new_cluster = cluster | {grid_id}
            zscore = Moran_Local(gdf.loc[new_cluster, COLS].values, w).Is[grid_id]
            similarity_scores[grid_id] = zscore
    return similarity_scores

In [10]:
gdf.explore()